In [ ]:
# Copyright 2025 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Automating Contract Analysis for Risk and Compliance

This notebook demonstrates a powerful, scalable solution for automating the analysis of legal contracts using Google BigQuery and Vertex AI's Gemini models. It showcases how to transform thousands of unstructured text documents stored in Google Cloud Storage into a structured, queryable, and insightful dataset directly within BigQuery. The process involves using SQL functions to extract key information, assess risk, generate summaries, and finally, visualize the results to enable data-driven decision-making for legal and compliance teams.

**The Business Challenge**: For legal, procurement, and finance departments in large organizations, the sheer volume of legal contracts represents a significant operational bottleneck and a hidden source of risk. Manually reviewing thousands of vendor agreements, sales contracts, and partnership deals to identify risks, track obligations, and ensure compliance is an expensive, slow, and notoriously error-prone process. A single missed renewal date can lead to unwanted service continuation, and an overlooked non-standard liability clause can expose the company to millions of dollars in unexpected risk.
The problem is one of scale and accessibility. Critical information is locked away in unstructured PDF and Word documents, making it impossible to perform portfolio-wide analysis. Legal teams cannot easily answer fundamental questions like, "How many of our contracts have a termination for convenience clause?" or "Which agreements are governed by a specific jurisdiction?" without a time-consuming manual review. This reactive approach prevents the business from proactively managing its contractual risks and obligations.


**The Data Science Approach**: This use case transforms a qualitative, manual task into a quantitative, automated one by leveraging Gemini models directly within BigQuery. By treating unstructured text documents in Cloud Storage as if they were structured data, we can use the power and familiarity of SQL to perform large-scale analysis.
The workflow begins by creating a remote model in BigQuery that points to a Gemini endpoint in Vertex AI. We then use the AI.GENERATE_TABLE function, providing it with a target schema and a natural language prompt. This prompt instructs the model on exactly what information to extract from each contract, such as "Identify the governing law," "Extract the contract's effective date and termination date," and "Summarize the limitation of liability clause."
The model processes each document and populates a new, structured BigQuery table based on the defined schema. This effectively turns a repository of thousands of legal documents into a queryable database. Data scientists and analysts can then use standard SQL to instantly find patterns, identify anomalies, and aggregate risk exposure across the entire contract portfolio. This enables them to build dashboards that monitor key clauses, set up alerts for upcoming renewal dates, and provide leadership with a comprehensive, data-driven view of the company's contractual landscape.


### High-level Flow

The notebook follows these key steps:

1. **Prerequisites and Setup**: Installs the necessary Python libraries and configures Google Cloud settings, including Project ID and region.
2. **Accessing Contract Data in BigQuery**: Creates a BigQuery external table that directly reads raw contract text files from a public Google Cloud Storage (GCS) bucket. This makes the unstructured data accessible via SQL without needing a separate data loading step.
3. **Enabling Generative AI in BigQuery**: Establishes a connection between BigQuery and Vertex AI and then creates a remote model that points to the Gemini 2.5 Flash model. This allows you to invoke the generative AI model using SQL functions.
4. **Transforming Text into Structured Data (AI.GENERATE_TABLE)**: Executes a single, powerful SQL query that feeds the raw text of each contract to the Gemini model. A detailed prompt instructs the model to parse the document and return structured information, which is used to automatically create a new, structured BigQuery table.
5. **Enriching Data with AI Summaries (AI.GENERATE)**: Builds upon the newly structured data by generating a concise executive summary for each contract. This step combines the extracted fields and the original text to create a high-quality, context-aware summary.
6. **Analyzing and Visualizing Insights**: With the contract data now fully structured and enriched, standard SQL queries are used to perform analysis. The results are then visualized using Plotly to create an interactive dashboard that answers key business questions, such as:
  * What is the distribution of different contract types?
  * Which contracts have the highest risk scores?
  * What is the average risk score per contract category?
  * How many contracts fall under specific legal jurisdictions?
  * Which contracts are expiring soon?

### Dataset: Real-World Commercial Contracts

This notebook uses a collection of full-text commercial agreements derived from the **Atticus Open Contract Dataset (AOK-Beta)**. This dataset is an excellent resource for our use case because it contains thousands of complex, real-world legal documents, mirroring the challenges that legal and compliance teams face daily.

*   **Data Source:** The raw contract text files are hosted in a public Google Cloud Storage (GCS) bucket. In this notebook, we will create a BigQuery external table to read this data directly, demonstrating how you can analyze documents in place without a complex data-loading pipeline.
*   **Content:** The dataset includes a diverse range of commercial agreements, making it ideal for testing the Gemini model's ability to parse and understand varied legal language and contract structures.
*   **Scalability:** The solution demonstrated here is fully scalable. While we process a large number of documents, the same architecture can be applied to analyze repositories containing tens of thousands or even millions of your own contracts stored in GCS.

## 0. Setup and Prerequisites

Before we begin, we need to set up our environment, install necessary libraries, and configure Google Cloud authentication.

### 0.1. Install Libraries

In [ ]:
%pip install --upgrade google-cloud-bigquery google-cloud-bigquery-storage google-auth pandas db-dtypes pyarrow plotly 

In [3]:
import os
import zipfile
import pandas as pd
from google.cloud import bigquery
import plotly.express as px
import plotly.graph_objects as go
import json
import sys

print("Libraries imported successfully!")

Libraries imported successfully!


### 0.2. Configure Google Cloud

#### Authenticate with Google Cloud:
* If you are using Google Colab, the following cell will handle authentication.
* If running locally, you should have the gcloud CLI installed and authenticated (gcloud auth application-default login).

In [4]:
# Authenticate user
if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user()

### 0.3. Set Your Project Variables:

* IMPORTANT: Replace the placeholder values with your own Google Cloud Project ID, the region you want to work in, and a name for your BigQuery dataset.

In [6]:
# @title Configuration
PROJECT_ID = "gcp-project-placeholder"  # @param {type:"string"}
REGION = "region-placeholder"            # @param {type:"string"} - Gemini 2.5 models are available here
DATASET_ID = "contract_analysis"  # @param {type:"string"}
RAW_TABLE_ID = "raw_contracts" # @param {type:"string"}
EXTRACTED_TABLE_ID = "extracted_contract_insights" # @param {type:"string"}
FINAL_TABLE_ID = "final_contract_analysis" # @param {type:"string"}
MODEL_ID = "gemini_2_5_flash" # @param {type:"string"}
MODEL_VERSION = "gemini-2.5-flash" # @param {type:"string"}
CONNECTION_ID = "vertex-ai-connection"
CONNECTION_CLOUD_RESOURCE_ID = "cloud-resource-connection"

GCS_PATH='gs://dataproc-metastore-public-binaries/cuad_v1/full_contract_txt/' # @param {type:"string"}

# Construct the full dataset reference
dataset_ref = f"{PROJECT_ID}.{DATASET_ID}"

# Initialize the BigQuery client
bq_client = bigquery.Client(project=PROJECT_ID)

## 1. Accessing Unstructured Contracts in BigQuery

To begin, we need to make our raw contract files—which are stored as text files in a public Google Cloud Storage (GCS) bucket—accessible to BigQuery. Instead of a traditional ETL process, we will create a BigQuery **external table**. This creates a metadata pointer to the files in GCS, allowing us to query them directly as if they were a native BigQuery table.

To grant BigQuery the necessary permissions to read from the GCS bucket, we first create a `CLOUD_RESOURCE` connection.

### 1.1 Create CLOUD_RESOURCE bq connection

In [ ]:
!bq mk --connection \
    --location={REGION} \
    --project_id={PROJECT_ID} \
    --connection_type=CLOUD_RESOURCE \
    {CONNECTION_CLOUD_RESOURCE_ID}

In [ ]:
bq_connection_show_json_txt = !bq show --connection --project_id={PROJECT_ID} --location={REGION} --format=prettyjson {CONNECTION_CLOUD_RESOURCE_ID}
bq_connection_show_json = json.loads("\n".join(bq_connection_show_json_txt))
connection_sa = bq_connection_show_json['cloudResource']['serviceAccountId']
print(f"Service Account: {connection_sa}")

In [ ]:
!gcloud projects add-iam-policy-binding {PROJECT_ID} \
    --member="serviceAccount:{connection_sa}" \
    --role="roles/storage.objectViewer"

### 1.2 Create bq object table

In [ ]:
create_metadata_gcs_external = f"""
CREATE OR REPLACE EXTERNAL TABLE `{PROJECT_ID}.{DATASET_ID}.{RAW_TABLE_ID}`
WITH CONNECTION `{PROJECT_ID}.{REGION}.{CONNECTION_CLOUD_RESOURCE_ID}`
OPTIONS (
  object_metadata = 'SIMPLE',
  uris = ['{GCS_PATH}*']
);
"""
print(create_metadata_gcs_external)
print("Creating GCS external metadata table in BigQuery...")
job = bq_client.query(create_metadata_gcs_external)
job.result() # Wait for the job to complete
print(f"✅ External metadata '{RAW_TABLE_ID}' created successfully.")

# Display a sample of the newly created structured data
print("\nSample of extracted data:")
df_extracted = bq_client.query(f"SELECT * FROM `{PROJECT_ID}.{DATASET_ID}.{RAW_TABLE_ID}` LIMIT 5").to_dataframe()
display(df_extracted)

## 2. Enabling Generative AI in BigQuery

To use Gemini's capabilities from within BigQuery, we must first create a connection to the Vertex AI service and then register a remote model.

*   A **Vertex AI Connection** is a BigQuery resource that enables SQL queries to securely interact with external services like Vertex AI.
*   A **Remote Model** in BigQuery acts as a proxy or a pointer to a specific pre-trained model hosted on Vertex AI, such as `gemini-2.5-flash`.

Once configured, we can call this model using functions like `AI.GENERATE()` directly in our SQL.

### 2.1. Create a Vertex AI Connection

This connection allows BigQuery to securely call services in Vertex AI.

In [ ]:
!bq mk --connection \
    --location={REGION} \
    --project_id={PROJECT_ID} \
    --connection_type=CLOUD_RESOURCE \
    {CONNECTION_ID}

Grant the 'Vertex AI User' role to the service account

In [ ]:
bq_connection_show_json_txt = !bq show --connection --project_id={PROJECT_ID} --location={REGION} --format=prettyjson {CONNECTION_ID}
bq_connection_show_json = json.loads("\n".join(bq_connection_show_json_txt))
connection_sa = bq_connection_show_json['cloudResource']['serviceAccountId']
print(f"Service Account: {connection_sa}")

In [ ]:
!gcloud projects add-iam-policy-binding {PROJECT_ID} \
    --member="serviceAccount:{connection_sa}" \
    --role="roles/aiplatform.user"

### 2.2. Create the Remote Model

This SQL statement creates a "model" in BigQuery that acts as a proxy to the gemini-2.5-flash model.

In [ ]:
create_model_sql = f"""
CREATE OR REPLACE MODEL `{dataset_ref}.{MODEL_ID}`
  REMOTE WITH CONNECTION `{PROJECT_ID}.{REGION}.{CONNECTION_ID}`
  OPTIONS (
    endpoint = '{MODEL_VERSION}'
  );
"""

print("Creating remote model in BigQuery...")
job = bq_client.query(create_model_sql)
job.result() # Wait for the job to complete
print(f"✅ Model '{MODEL_ID}' created successfully.")

## 3. Transforming Unstructured Text into Structured Data

This step is the core of our solution. We will use the `AI.GENERATE_TABLE` function to read each contract's raw text and transform it into a fully structured table in a single operation.

We provide the function with a detailed prompt that acts as a set of instructions for the Gemini model. The prompt defines the model's role (a legal analyst), the input it will receive (the contract text), and the desired output schema (the specific fields to extract, like `ContractTitle`, `RiskScore`, etc.). BigQuery handles the parallel execution, calling the model for each document and assembling the results into a new, permanent table.

### 3.1. Define the Extraction Prompt

A good prompt is crucial for getting accurate and well-formatted results.

In [ ]:
# This detailed prompt instructs the model on its role, the input, the desired output format,
# and the specific fields to extract.
extraction_prompt = """
You are a highly skilled legal analyst AI. Your task is to analyze a legal contract and extract key information.

Analyze the contract provided in the "prompt" column, which contains the full text of the agreement.

From the contract text, extract the following fields and return them as a single, valid JSON object. If a value cannot be found, use "Not Found" for string fields and null for numeric or date fields.

- **ContractTitle**: The main title of the agreement (e.g., "Master Services Agreement", "Lease Agreement").
- **AgreementDate**: The stated date of the agreement, often found at the beginning. Format as YYYY-MM-DD.
- **EffectiveDate**: The date the agreement becomes effective. This may be different from the agreement date. Format as YYYY-MM-DD.
- **ExpirationDate**: The date the agreement terminates or expires. Format as YYYY-MM-DD.
- **Parties**: A list of all named parties (companies, individuals) in the agreement.
- **GoverningLaw**: The state or country whose laws will govern the agreement (e.g., "State of Delaware", "England and Wales").
- **Locations**: A list of any specific cities, states, or physical addresses mentioned as relevant locations for notices, services, or jurisdiction.
- **AgreementCategory**: Classify the contract into one of the following categories: "Services", "Lease", "Employment", "Loan", "Partnership", "NDA", "Software License", "Other".
- **RiskAnalysis**: Provide a brief (1-2 sentence) analysis of the potential risks for the primary party, based on the text. Consider terms related to liability, termination, and confidentiality.
- **RiskScore**: Assign a risk score from 1 (low risk) to 10 (high risk) based on your analysis. This should be a numeric value.
- **IsConfidential**: A boolean value (true/false) indicating if the agreement contains explicit confidentiality clauses.
- **RenewalTerms**: A brief description of the renewal terms (e.g., "Auto-renews annually", "No renewal clause", "Mutual agreement required").
"""

### 3.2. Run the AI.GENERATE_TABLE Query

This query will read from our raw text table, apply the prompt using the Gemini model, and create a new table with the structured results.


In [ ]:
# For the demo, we'll process a small subset (LIMIT 10).
# On a full run, you would remove the LIMIT clause.
extract_query = f"""
CREATE OR REPLACE TABLE `{dataset_ref}.{EXTRACTED_TABLE_ID}` AS (
  SELECT
    ml_generate_table_result.*
  FROM
    AI.GENERATE_TABLE(
      MODEL `{dataset_ref}.{MODEL_ID}`,
      (
        SELECT
          uri,
          data,
          -- Pass our detailed prompt to the model for each row
          '''{extraction_prompt}''' AS prompt
        FROM
          {dataset_ref}.{RAW_TABLE_ID}
        -- LIMIT 10 -- Limiting for the demo. Remove for full run.
      ),
      STRUCT(
         '''
        ContractTitle STRING,
        AgreementDate STRING,
        EffectiveDate STRING,
        ExpirationDate STRING,
        Parties ARRAY<STRING>,
        GoverningLaw STRING,
        Locations ARRAY<STRING>,
        AgreementCategory STRING,
        RiskAnalysis STRING,
        RiskScore INT64,
        IsConfidential BOOL,
        RenewalTerms STRING
        ''' AS output_schema,
        0.1 AS temperature,          -- Lower temperature for more deterministic output
        4096 AS max_output_tokens
      )
    ) AS ml_generate_table_result
);
"""
print(extract_query)
print("Starting information extraction with AI.GENERATE_TABLE...")
print("This may take a few minutes depending on the number of documents...")
job = bq_client.query(extract_query)
job.result() # Wait for the job to complete
print("✅ Extraction complete! Structured data is now in the `extracted_contract_insights` table.")

# Display a sample of the newly created structured data
print("\nSample of extracted data:")
df_extracted = bq_client.query(f"SELECT * FROM `{dataset_ref}.{EXTRACTED_TABLE_ID}` LIMIT 5").to_dataframe()
display(df_extracted)

## 4. Data Enrichment with AI-Generated Summaries

Now that we have structured fields extracted from each contract, we can enrich our dataset with high-quality, context-aware summaries. We'll use the `AI.GENERATE` function to create an `ExecutiveSummary` column.

For each contract, we will construct a new prompt that includes both the original text and the key details we just extracted (like Title, Parties, and Risk Score). This provides the model with rich context, enabling it to generate a more accurate and relevant summary focused on the most important aspects of the agreement.

### 4.1. Run the Summarization Query

This query creates our final table, including all previously extracted data plus the new AI-generated summary.

In [ ]:
# This query reads from our extracted data table, formats the summarization prompt,
# calls the model, and creates the final enriched table.
summarize_query = f"""
CREATE OR REPLACE TABLE `{dataset_ref}.{FINAL_TABLE_ID}` AS (
SELECT
    *,
    AI.GENERATE(
    ('Based on the full contract text and the following extracted key details, please generate a concise executive summary.',
     'The summary should be no more than 3-4 bullet points and should highlight the purpose of the agreement, the key obligations of the main parties, and the agreement duration or termination conditions.',
     'Based on the full contract text and the following extracted key details, please generate a concise executive summary.',
     '**Extracted Details:**',
     '- Title: ',t.ContractTitle,
     '- Category: ', t.AgreementCategory,
     '- Parties: %s', ARRAY_TO_STRING(t.Parties,','),
     '- Effective Date: ', CAST(t.EffectiveDate AS STRING),
     '- Expiration: ', CAST(t.ExpirationDate AS STRING),
     '- Governing Law: ', t.GoverningLaw,
     '- Risk Score:', CAST(t.RiskScore AS STRING),
     '**Full Contract Text:**', CAST(t.data AS STRING)
     ),
    connection_id => '{REGION}.{CONNECTION_ID}',
    endpoint => '{MODEL_VERSION}').result as ExecutiveSummary
  FROM `{dataset_ref}.{EXTRACTED_TABLE_ID}` AS t
  );
"""
print(summarize_query)
print("Generating executive summaries with AI.GENERATE...")
job = bq_client.query(summarize_query)
job.result() # Wait for completion
print("✅ Summarization complete! Final data is in the `final_contract_analysis` table.")

In [ ]:
# Display a sample of the final data, including the summary
print("\nSample of final data with executive summary:")
df_final = bq_client.query(f"SELECT uri, ContractTitle, RiskScore, ExecutiveSummary FROM `{dataset_ref}.{FINAL_TABLE_ID}` WHERE  ContractTitle <> 'Not Found' LIMIT 5").to_dataframe()
display(df_final)

## 5. Gaining Actionable Insights from Contract Data

With our contract data fully structured and enriched in BigQuery, we can now perform powerful analysis using standard SQL. This allows us to move from analyzing single documents to understanding our entire contract portfolio.

In this section, we will query our final table to answer key business questions and use Plotly to create interactive visualizations for an at-a-glance understanding of our contract landscape.

### 5.1. SQL Analysis

Let's ask some business-relevant questions.

In [7]:
# --- Query 1: What is the distribution of contract categories? ---
print("--- Analysis 1: Contract Category Distribution ---")
q1_sql = f"""
    SELECT
        AgreementCategory,
        COUNT(*) as contract_count
    FROM `{dataset_ref}.{FINAL_TABLE_ID}`
    WHERE AgreementCategory IS NOT NULL AND AgreementCategory != 'Not Found'
    GROUP BY AgreementCategory
    ORDER BY contract_count DESC;
"""
df_categories = bq_client.query(q1_sql).to_dataframe()
display(df_categories)


# --- Query 2: Which contracts have the highest risk score? ---
print("\n--- Analysis 2: Top 5 High-Risk Contracts ---")
q2_sql = f"""
    SELECT
        uri,
        ContractTitle,
        RiskScore,
        RiskAnalysis
    FROM `{dataset_ref}.{FINAL_TABLE_ID}`
    WHERE RiskScore IS NOT NULL
    ORDER BY RiskScore DESC
    LIMIT 5;
"""
df_high_risk = bq_client.query(q2_sql).to_dataframe()
display(df_high_risk)


# --- Query 3: How many contracts are governed by Delaware law? ---
print("\n--- Analysis 3: Contracts Governed by Delaware Law ---")
q3_sql = f"""
    SELECT
        COUNT(*) as count_delaware_law
    FROM `{dataset_ref}.{FINAL_TABLE_ID}`
    WHERE GoverningLaw LIKE '%Delaware%';
"""
df_delaware = bq_client.query(q3_sql).to_dataframe()
display(df_delaware)

# --- Query 4: Find contracts expiring soon ---
print("\n--- Analysis 4: Contracts Expiring in the Next Year ---")
# This query identifies contracts with an expiration date within the next 365 days.
# Note: This requires the ExpirationDate to be correctly parsed. We use PARSE_DATE assuming a 'YYYY-MM-DD' format.
q4_sql = f"""
SELECT
  uri,
  ContractTitle,
  ExpirationDate,
  RiskScore
FROM `{dataset_ref}.{FINAL_TABLE_ID}`
WHERE ExpirationDate IS NOT NULL AND ExpirationDate != 'Not Found'
  AND SAFE.PARSE_DATE('%Y-%m-%d', ExpirationDate) BETWEEN CURRENT_DATE() AND DATE_ADD(CURRENT_DATE(), INTERVAL 1 YEAR)
ORDER BY ExpirationDate ASC;
"""
df_expiring = bq_client.query(q4_sql).to_dataframe()
display(df_expiring)

# --- Query 5: Analyze risk by category ---
print("\n--- Analysis 5: Average Risk Score by Contract Category ---")
# This query calculates the average risk score for each contract category, helping to identify which types of agreements
# generally carry higher risk.
q5_sql = f"""
SELECT
  AgreementCategory,
  COUNT(*) as contract_count,
  ROUND(AVG(RiskScore), 2) as average_risk
FROM `{dataset_ref}.{FINAL_TABLE_ID}`
WHERE AgreementCategory IS NOT NULL AND AgreementCategory != 'Not Found' AND RiskScore IS NOT NULL
GROUP BY AgreementCategory
ORDER BY average_risk DESC;
"""
df_avg_risk = bq_client.query(q5_sql).to_dataframe()
display(df_avg_risk)

--- Analysis 1: Contract Category Distribution ---


,AgreementCategory,contract_count
0,Services,306
1,Software License,173
2,Other,20



--- Analysis 2: Top 5 High-Risk Contracts ---


,uri,ContractTitle,RiskScore,RiskAnalysis
0,gs://dataproc-metastore-public-binaries/cuad_v...,SOFTWARE LICENSE AGREEMENT,8,"For the Licensee, the limitation of liability ..."
1,gs://dataproc-metastore-public-binaries/cuad_v...,SOFTWARE LICENSE AGREEMENT,8,The Licensee faces significant risk due to bro...
2,gs://dataproc-metastore-public-binaries/cuad_v...,SOFTWARE LICENSE AGREEMENT,8,"For the Licensee, the primary risk is the sign..."
3,gs://dataproc-metastore-public-binaries/cuad_v...,SOFTWARE LICENSE AGREEMENT,8,"For the Licensee, significant risks include th..."
4,gs://dataproc-metastore-public-binaries/cuad_v...,SOFTWARE LICENSE AGREEMENT,8,"For the Licensee, significant risks include th..."



--- Analysis 3: Contracts Governed by Delaware Law ---


,count_delaware_law
0,424



--- Analysis 4: Contracts Expiring in the Next Year ---


,uri,ContractTitle,ExpirationDate,RiskScore
0,gs://dataproc-metastore-public-binaries/cuad_v...,MASTER SERVICES AGREEMENT,2025-10-15,6
1,gs://dataproc-metastore-public-binaries/cuad_v...,MASTER SERVICES AGREEMENT,2025-10-25,5
2,gs://dataproc-metastore-public-binaries/cuad_v...,MASTER SERVICES AGREEMENT,2025-10-26,6
3,gs://dataproc-metastore-public-binaries/cuad_v...,Master Services Agreement,2025-10-26,6
4,gs://dataproc-metastore-public-binaries/cuad_v...,SOFTWARE LICENSE AGREEMENT,2025-12-31,3
...,...,...,...,...
408,gs://dataproc-metastore-public-binaries/cuad_v...,MASTER SERVICES AGREEMENT,2026-01-01,8
409,gs://dataproc-metastore-public-binaries/cuad_v...,Software License Agreement,2026-01-14,5
410,gs://dataproc-metastore-public-binaries/cuad_v...,SOFTWARE LICENSE AGREEMENT,2026-01-14,7
411,gs://dataproc-metastore-public-binaries/cuad_v...,SOFTWARE LICENSE AGREEMENT,2026-01-15,7



--- Analysis 5: Average Risk Score by Contract Category ---


,AgreementCategory,contract_count,average_risk
0,Software License,173,6.23
1,Services,306,5.36
2,Other,20,0.20


### 5.2. Visualization with Plotly

Let's turn the results from our analysis into interactive charts.

#### Visualization 1: What is our overall contract portfolio mix?

This bar chart shows the number of contracts in each category, giving a quick overview of the types of agreements in our dataset.

In [8]:
# Create a bar chart for contract category distribution
fig_categories = px.bar(
    df_categories,
    x='AgreementCategory',
    y='contract_count',
    title='Distribution of Contract Categories',
    labels={'AgreementCategory': 'Contract Category', 'contract_count': 'Number of Contracts'},
    color='AgreementCategory',
    text_auto=True  # Display the count on each bar
)

# Improve layout
fig_categories.update_layout(
    xaxis_title="Contract Category",
    yaxis_title="Number of Contracts",
    showlegend=False
)

fig_categories.show()

#### Visualization 2: Which contracts require immediate attention?

This chart highlights the contracts that the AI has identified as having the highest risk scores, allowing legal teams to immediately prioritize their review process.

In [9]:
# Create a bar chart for the highest-risk contracts
fig_high_risk = px.bar(
    df_high_risk,
    x='ContractTitle',
    y='RiskScore',
    title='Top 5 High-Risk Contracts',
    labels={'ContractTitle': 'Contract Title', 'RiskScore': 'Risk Score (1-10)'},
    color='RiskScore',
    color_continuous_scale=px.colors.sequential.Reds,
    hover_data=['uri', 'RiskAnalysis'] # Show more details on hover
)

# Improve layout
fig_high_risk.update_layout(
    xaxis_title="Contract Title",
    yaxis_title="Risk Score",
    coloraxis_showscale=False
)
fig_high_risk.update_xaxes(categoryorder='total descending') # Order by risk score

fig_high_risk.show()

#### Visualization 3: Which contract types are inherently riskier?

By visualizing the average risk score per category, we can identify which types of agreements tend to carry more inherent risk across our portfolio.

In [10]:
# --- Query to get the data for the chart ---
q_avg_risk_sql = f"""
SELECT
  AgreementCategory,
  ROUND(AVG(RiskScore), 2) as average_risk_score
FROM `{dataset_ref}.{FINAL_TABLE_ID}`
WHERE AgreementCategory IS NOT NULL AND AgreementCategory != 'Not Found' AND RiskScore IS NOT NULL
GROUP BY AgreementCategory
ORDER BY average_risk_score DESC;
"""
df_avg_risk = bq_client.query(q_avg_risk_sql).to_dataframe()


# --- Create the bar chart ---
fig_avg_risk = px.bar(
    df_avg_risk,
    x='AgreementCategory',
    y='average_risk_score',
    title='Average Risk Score by Contract Category',
    labels={'AgreementCategory': 'Contract Category', 'average_risk_score': 'Average Risk Score'},
    color='average_risk_score',
    color_continuous_scale=px.colors.sequential.OrRd,
    text_auto=True
)

# Improve layout
fig_avg_risk.update_layout(
    xaxis_title="Contract Category",
    yaxis_title="Average Risk Score"
)

fig_avg_risk.show()

#### Visualization 4: What is the primary legal jurisdiction for our contracts?

This pie chart illustrates the jurisdictional landscape, showing which state's or country's laws govern the most agreements.

In [11]:
# --- Query to get the distribution of governing laws ---
q_law_sql = f"""
SELECT
  GoverningLaw,
  COUNT(*) as contract_count
FROM `{dataset_ref}.{FINAL_TABLE_ID}`
WHERE GoverningLaw IS NOT NULL AND GoverningLaw != 'Not Found'
GROUP BY GoverningLaw
ORDER BY contract_count DESC
LIMIT 10; -- Limit to top 10 for clarity
"""
df_law = bq_client.query(q_law_sql).to_dataframe()


# --- Create the pie chart ---
fig_law = px.pie(
    df_law,
    names='GoverningLaw',
    values='contract_count',
    title='Top 10 Governing Law Jurisdictions',
    hole=0.3 # Create a donut chart
)

fig_law.update_traces(textposition='inside', textinfo='percent+label')
fig_law.show()

#### Visualization 5: What is the overall risk profile of our contracts?

This histogram visualizes the distribution of risk scores across all contracts, helping us understand whether our portfolio leans toward low-risk or high-risk agreements.


In [12]:
# --- Query to get all valid risk scores ---
q_risk_dist_sql = f"""
SELECT
  RiskScore
FROM `{dataset_ref}.{FINAL_TABLE_ID}`
WHERE RiskScore IS NOT NULL;
"""
df_risk_dist = bq_client.query(q_risk_dist_sql).to_dataframe()

# --- Create the histogram ---
fig_risk_dist = px.histogram(
    df_risk_dist,
    x='RiskScore',
    nbins=10, # We want a bin for each score from 1 to 10
    title='Overall Risk Score Distribution',
    labels={'RiskScore': 'Risk Score (1-10)', 'count': 'Number of Contracts'},
    color_discrete_sequence=['#ef553b'] # Use a risk-associated color
)

fig_risk_dist.update_layout(
    xaxis_title="Risk Score",
    yaxis_title="Number of Contracts",
    bargap=0.1
)
fig_risk_dist.update_traces(marker_line_width=1, marker_line_color="black")


fig_risk_dist.show()

#### Visualization 6: How many of our contracts contain confidentiality clauses?

This donut chart reveals the proportion of contracts with explicit confidentiality clauses—a critical metric for compliance and data governance.

In [13]:
# --- Query to count contracts by confidentiality status ---
q_confidential_sql = f"""
SELECT
  IsConfidential,
  COUNT(*) as contract_count
FROM `{dataset_ref}.{FINAL_TABLE_ID}`
WHERE IsConfidential IS NOT NULL
GROUP BY IsConfidential;
"""
df_confidential = bq_client.query(q_confidential_sql).to_dataframe()

# Map boolean to more descriptive labels for the chart
df_confidential['Label'] = df_confidential['IsConfidential'].map({True: 'Contains Confidentiality Clause', False: 'No Confidentiality Clause'})

# --- Create the donut chart ---
fig_confidential = px.pie(
    df_confidential,
    names='Label',
    values='contract_count',
    title='Proportion of Contracts with Confidentiality Clauses',
    hole=0.4, # This creates the donut shape
    color_discrete_map={
        'Contains Confidentiality Clause':'#636EFA',
        'No Confidentiality Clause':'#a9aef7'
    }
)

fig_confidential.update_traces(textinfo='percent+label', pull=[0, 0.05])
fig_confidential.show()

#### Visualization 7: What does our contract renewal pipeline look like?

This timeline visualizes the lifespan of each contract from its effective date to its expiration date. Color-coded by risk, this view is invaluable for tracking active agreements, planning renewals, and identifying high-risk contracts that are nearing their end date.

In [15]:
# --- Query to get contract dates and risk scores ---
# We'll parse the dates and filter for valid entries. We limit to 15 for a clean visual.
q_timeline_sql = f"""
SELECT
  ContractTitle,
  RiskScore,
  PARSE_DATE('%Y-%m-%d', EffectiveDate) as StartDate,
  PARSE_DATE('%Y-%m-%d', ExpirationDate) as EndDate
FROM `{dataset_ref}.{FINAL_TABLE_ID}`
WHERE
  EffectiveDate IS NOT NULL AND EffectiveDate != 'Not Found' AND
  ExpirationDate IS NOT NULL AND ExpirationDate != 'Not Found' AND
  -- Ensure dates are in a valid format for parsing
  LENGTH(EffectiveDate) = 10 AND LENGTH(ExpirationDate) = 10
ORDER BY EndDate DESC
LIMIT 15;
"""
df_timeline = bq_client.query(q_timeline_sql).to_dataframe()


# --- Create the timeline chart ---
fig_timeline = px.timeline(
    df_timeline,
    x_start="StartDate",
    x_end="EndDate",
    y="ContractTitle",
    color="RiskScore",
    title="Contract Lifecycle Timeline (Color-Coded by Risk Score)",
    labels={'ContractTitle': 'Contract', 'RiskScore': 'Risk Score'},
    color_continuous_scale=px.colors.sequential.YlOrRd
)

# Improve layout
fig_timeline.update_yaxes(autorange="reversed") # Show newest expirations at the top
fig_timeline.update_layout(
    xaxis_title="Date",
    yaxis_title="Contract"
)

fig_timeline.show()